## 数据探索

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 为了方便叙述建模流程，这里准备了两个脱敏数据集：一个训练集一个测试集
train = pd.read_csv('imb_train.csv')
test = pd.read_csv('imb_test.csv')

print(f'训练集数据长度：{len(train)}，测试集数据长度：{len(test)}')
train.sample(3)

# 参数解释
## X1 ~ X5：自变量，cls：因变量 care life of science - 科学关爱生命 0-不得病，1-得病

训练集数据长度：14000，测试集数据长度：6000


,X1,X2,X3,X4,X5,cls
318,0.255582,-0.578199,-0.504863,-0.963650,-1.015626,0
1513,-1.523422,1.583999,0.094002,-2.276259,-1.293283,0
10025,1.099701,-1.785462,1.767452,-1.121677,-1.599187,0


In [3]:
# 查看测试集与训练集的因变量分类情况
print('训练集中，因变量 cls 分类情况：')
print(train['cls'].agg(['value_counts']).T)
print('='*55 + '\n')

print('测试集中，因变量 cls 分类情况：')
print(test['cls'].agg(['value_counts']).T)
# 可知训练集和测试集中的占比少的类别 1 实在是太少了，比较严重的不平衡

训练集中，因变量 cls 分类情况：
                  0    1
value_counts  13644  356

测试集中，因变量 cls 分类情况：
                 0    1
value_counts  5848  152


In [4]:
# 另一种探索方法，直接使用 collection 库里面的 Counter 函数
from collections import Counter
print('训练集中因变量 cls 分类情况：{}'.format(Counter(train['cls'])))
print('测试集因变量 cls 分类情况：{}'.format(Counter(test['cls'])))

训练集中因变量 cls 分类情况：Counter({0: 13644, 1: 356})
测试集因变量 cls 分类情况：Counter({0: 5848, 1: 152})


## 不同的抽样方法对训练集进行处理
测试集是不做任何处理的！！保留严峻的比例考验来测试模型。

训练模型时用到的数据才是经过处理的，0-1 比例再 1：1 ~ 1：10 之间

### 拆分因变量与自变量

In [5]:
y_train = train['cls'];        y_test = test['cls']
X_train = train.loc[:, :'X5'];  X_test = test.loc[:, :'X5']

In [6]:
X_train.sample(), y_train[:1] 

(            X1        X2        X3       X4        X5
 9382 -1.191287  1.363136 -0.705131 -1.24394 -0.520264, 0    0
 Name: cls, dtype: int64)

### 抽样的几种方法
- Random Over Sampling：随机过抽样
- SMOTE 方法过抽样
- SMOTETomek 综合抽样

In [7]:
# imblearn：imbalance learning 这个包 pip install imblearn 安装一下即可
from imblearn.over_sampling import RandomOverSampler

print('不经过任何采样处理的原始 y_train 中的分类情况：{}'.format(Counter(y_train)))

# 采样策略 sampling_strategy = 'auto' 的 auto 默认抽成 1：1，
 ## 如果想要另外的比例如杰克所说的 1：5，甚至底线 1:10，需要根据文档自行调整参数
 ## 文档：https://imbalanced-learn.readthedocs.io/en/stable/generated/imblearn.over_sampling.RandomOverSampler.html
# 先定义好好，未开始正式训练拟合
ros = RandomOverSampler(random_state=0, sampling_strategy='auto') 
X_ros, y_ros = ros.fit_sample(X_train, y_train)
print('随机过采样后，训练集 y_ros 中的分类情况：{}'.format(Counter(y_ros)))

# 同理，SMOTE 的步骤也是如此
from imblearn.over_sampling import SMOTE
sos = SMOTE(random_state=0)
X_sos, y_sos = sos.fit_sample(X_train, y_train)
print('SMOTE过采样后，训练集 y_sos 中的分类情况：{}'.format(Counter(y_sos)))

# 同理，综合采样（先过采样再欠采样）
## # combine 表示组合抽样，所以 SMOTE 与 Tomek 这两个英文单词写在了一起
from imblearn.combine import SMOTETomek
kos = SMOTETomek(random_state=0)  # 综合采样
X_kos, y_kos = kos.fit_sample(X_train, y_train)
print('综合采样后，训练集 y_kos 中的分类情况：{}'.format(Counter(y_kos)))
# 不难两种过采样方法都将原来 y_train 中的占比少的分类 1 提到了与 0 数量一致的情况
# 但因为综合采样在过采样后会使用欠采样，所以数量会稍微少一点点

不经过任何采样处理的原始 y_train 中的分类情况：Counter({0: 13644, 1: 356})
随机过采样后，训练集 y_ros 中的分类情况：Counter({0: 13644, 1: 13644})
SMOTE过采样后，训练集 y_sos 中的分类情况：Counter({0: 13644, 1: 13644})
综合采样后，训练集 y_kos 中的分类情况：Counter({0: 13395, 1: 13395})


## 决策树建模

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [9]:
# 创建了决策树类，还并没有正式开始训练模型
clf = DecisionTreeClassifier(criterion='gini', random_state=1234)
# 梯度优化
param_grid = {'max_depth':[3, 4, 5, 6], 'max_leaf_nodes':[4, 6, 8, 10, 12]}
# cv 表示是创建一个类，还并没有开始训练模型
cv = GridSearchCV(clf, param_grid=param_grid, scoring='f1')

In [10]:
# 注意！！！！！！！！
## 这里的数据使用大有玄机
## 第一组数据 X，y_train 是没有经过任何操作的
## 第二组 ros 为随机过采样，第三组 sos 为 SMOTE 过采样，
## 最后一组 kos 则为综合采样
data = [[X_train, y_train],
        [X_ros, y_ros],
        [X_sos, y_sos],
        [X_kos, y_kos]]

In [11]:
for features, labels in data:
    cv.fit(features, labels) # 对四组数据分别做模型
    # 注意：X_test 是从来没被动过的，回应了理论知识：
     ## 使用比例优良的(1:1~1:10)训练集来训练模型，用残酷的(分类为1的仅有2%)测试集来考验模型
    predict_test = cv.predict(X_test) 
# 其实 recall 和 precision 的用处都不大，看 auc 即可
## recall：覆盖率，预测出分类为 0 且正确的，但本来数据集中分类为 0 的占比本来就很大
## 而且 recall 是以阈值为 0.5 来计算的，那我们就可以简单的认为预测的欺诈概率大于 0.5 就算欺诈了吗？还是说如果他的潜在欺诈概率只要超过 20% 就已经算为欺诈了呢？
## 另外：阈值也不是 0.01，正确确定阈值的方法应该是结合先验概率(1%)，在 1% ~ 50% 之间，根据公式
    print('auc:%.3f' %metrics.roc_auc_score(y_test, predict_test), 
          'recall:%.3f' %metrics.recall_score(y_test, predict_test),
          'precision:%.3f' %metrics.precision_score(y_test, predict_test))
    # 发现并不一定是综合采样就一定高分，毕竟每份数据集都有属于它自己的特征
    ## 不过一点都不处理的模型的 auc 是最低的

auc:0.747 recall:0.493 precision:0.987
auc:0.824 recall:0.783 precision:0.132
auc:0.819 recall:0.757 precision:0.143
auc:0.819 recall:0.757 precision:0.142


## 采用权重法

In [12]:
#2、采用改变样本权重的方法
param_grid2 = {'max_depth':[3, 4, 5, 6], 
               'max_leaf_nodes':[4, 6, 8, 10, 12], 
               'class_weight':[{0:1, 1:5}, {0:1, 1:10}, {0:1, 1:15}]}
# y=0:y=1 = 1:5/1:1-/1:15，逐个尝试，但需要注意的是，
 ## 这个权重比例并不是简单的将占比少的 y=1 的数据✖5，
 ## 而是在计算模型精度的时候这个权重才派上用场，即如果你预测错一个 y=1 的，
 ## 就算预测错 5/10/15 个
# clf 是已经定义好的决策树
cv2 = GridSearchCV(clf, param_grid=param_grid2, scoring='f1')

# 当使用权重法的时候是需要在原始的数据集上使用
cv2.fit(X_train, y_train)
predict_test2 = cv2.predict(X_test)

print('auc:%.3f' %metrics.roc_auc_score(y_test, predict_test2),
      'recall:%.3f' %metrics.recall_score(y_test, predict_test2),
      'precision:%.3f' %metrics.precision_score(y_test, predict_test2))
# 肯定比不做任何操作要好好，但不够使用算法来进行采样处理好。

cv2.best_params_

auc:0.806 recall:0.618 precision:0.740


{'class_weight': {0: 1, 1: 5}, 'max_depth': 4, 'max_leaf_nodes': 12}